In [393]:
import vk_api

from tqdm import tqdm
import regex
import pandas as pd


In [376]:
BUGURT_OWNER_ID = -57536014

ALLOWED_SYMBOLS_REGEX = """[\w:"\./, !?\-\(\)%–—♂'`\*«»(#БУГУРТ)(#достаточно)(#Антибугурт)\[\]\+]"""
SEPARATOR_REGEX = "\n( *@ *\n)+"
BUGURT_PATTERN = regex.compile("(({}+)({})?)+".format(ALLOWED_SYMBOLS_REGEX, SEPARATOR_REGEX))

In [398]:
def get_iter():
    """ Пример получения всех постов со стены """

    login, password = '', ''
    vk_session = vk_api.VkApi(login, password)

    try:
        vk_session.auth(token_only=True)
    except vk_api.AuthError as error_msg:
        print(error_msg)
        return

    tools = vk_api.VkTools(vk_session)

    """ VkTools.get_all позволяет получить все объекты со всех страниц.
        Соответственно get_all используется только если метод принимает
        параметры: count и offset.

        Например может использоваться для получения всех постов стены,
        всех диалогов, всех сообщений, etc.

        При использовании get_all сокращается количество запросов к API
        за счет метода execute в 25 раз.

        Например за раз со стены можно получить 100 * 25 = 2500, где
        100 - максимальное количество постов, которое можно получить за один
        запрос (обычно написано на странице с описанием метода)
    """

    return tools.get_all_slow_iter('wall.get', 100, {'owner_id': BUGURT_OWNER_ID})

In [399]:
good = 0
bad = 0

iterator = get_iter()

In [400]:
posts = []

for post in tqdm(iterator):
    text = post['text']
    if regex.fullmatch(BUGURT_PATTERN, text):
        good += 1
        text = regex.sub(SEPARATOR_REGEX, "[SEP]", text)
        posts.append(text)
    else:
        bad += 1

90594it [11:04, 136.38it/s]


In [ ]:
print("Good ratio:", good / (good + bad))
print("Total good:", good)

In [402]:
df = pd.DataFrame(posts, columns=["text"])
df["lines"] = df["text"].apply(lambda x: x.split("[SEP]"))

In [ ]:
df = df.drop("text", axis=1)

In [412]:
df.to_json("data.json", lines=True, orient="records")